### PROJET NLP - EXPLORATION DES FONCTIONS ANALYTIQUES DU LANGAGE NATUREL

In [33]:
# Décommenter les lignes suivantes et exécuter la cellule lors du premier lancement pour installer les dépendances :

# !pip install spacy
# !python -m spacy download fr_core_news_md


## Présentation de SpaCy

SpaCy est une bibliothèque de traitement automatique du langage naturel (NLP). Elle permet de faire de l'analyse grammaticale, de la reconnaissance d'entités nommées et du décodage de mots-clés.

In [34]:
import spacy
from spacy import displacy

Spacy propose plusieurs modèles pré-entrainés. Celui que nous utilisons ici est le modèle medium en français.

In [35]:
# Chargement du modèle pré-entrainé
nlp = spacy.load("fr_core_news_md")

SpaCy procède à l'analyse d'un texte brut en le convertissant en objet "Doc".
Ce Doc est décomposé en tokens (entités linguistiques). Ces tokens nous permettront d'extraire plusieurs informations textuelles.

In [36]:
# Définition d'une texte à analyse
texte_brut = "Je vais de Paris à Londres demain. Je quitte Bordeaux pour aller à Marseille. Je souhaite me rendre à Lyon depuis Toulouse pour acheter un iPhone chez Aple ou chez orenge avec mon ami Jean en mangeant une orange sur la route."

# Définition d'un objet de type Doc à partir du texte brut
doc = nlp(texte_brut)

In [37]:
# Affichage des tokens du Doc, de leur POS (Part Of Speech) et de leur DEP (dépendance, ou relation grammaticale avec d'autres token)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Je PRON nsubj
vais VERB ROOT
de ADP case
Paris PROPN obl:arg
à ADP case
Londres PROPN obl:mod
demain ADV advmod
. PUNCT punct
Je PRON nsubj
quitte VERB ROOT
Bordeaux PROPN obj
pour ADP mark
aller VERB advcl
à ADP case
Marseille PROPN obl:arg
. PUNCT punct
Je PRON nsubj
souhaite VERB ROOT
me PRON expl:comp
rendre VERB xcomp
à ADP case
Lyon PROPN obl:arg
depuis ADP case
Toulouse PROPN obl:mod
pour ADP mark
acheter VERB advcl
un DET det
iPhone NOUN obj
chez ADP case
Aple PROPN obl:arg
ou CCONJ cc
chez ADP case
orenge NOUN conj
avec ADP case
mon DET det
ami NOUN obl:mod
Jean PROPN appos
en ADP mark
mangeant VERB advcl
une DET det
orange NOUN obj
sur ADP case
la DET det
route NOUN nmod
. PUNCT punct


In [38]:
# Décomposition du Doc en une liste de phrases
sentence_spans = list(doc.sents)

# Affichage des dépandances entre les tokens de chaque phrase
displacy.render(sentence_spans, style="dep")


In [39]:
# Visualisation des entités nommées (Named Entity Recognition ou NER, ce qui permet d'identifier des entités comme des personnes, des lieux, des organisations, etc.)
displacy.render(doc, style="ent")

Les graphiques précédents montrent que 
- les lieux dont clairement identifiés en tant que LOC
- la relation entre les prépositions et les lieux est bien établie
- le modèle possède une compréhension sémantique poussée (pas de confusion entre les fruits et les organisations) y compris en cas de fautes d'orthographes

## Mise en pratique pour l'analyse de lieux de départ et de lieu d'arrivée

### Méthodologie :
- Convertir le texte brut en Doc pour l'analyser
- Repérer les token de type LOC
- Identifier si la préposition indique un départ (de, depuis) ou une destination (à, vers)

In [40]:


# Définition des phrases d'exemple
phrases = [
    "Je vais de Paris à Londres demain.",
    "Je quitte Bordeaux pour aller à Marseille.",
    "Je souhaite me rendre à Lyon depuis Toulouse.",
    "Je mange de la glace à Paris.", # Phrase piège avec "de"
    "Je marche vers la mer.", # Phrase piège avec "vers"
    "Je n'ai pas mangé depuis ce matin." # Phrase piège avec "depuis"
]

# Fonction pour extraire les localisations et déterminer s'il s'agit d'un départ ou d'une destination
def extraire_localisations(text):
    doc = nlp(text)
    depart = None
    destination = None

    for token in doc:
        # Si le token est de type LOC, on analyse les tokens enfants pour déterminer s'il s'agit d'une préposition (case) et si oui, laquelle (de, à, depuis, etc.
        if token.ent_type_ in ["LOC"]:
            for subtoken in token.children :
                if(subtoken.dep_ == "case"):
                    # Si c'est une préposition de départ
                    if(subtoken.text in ["de", "depuis"]):
                        depart = token.text
                    # Si c'est une préposition de destination
                    elif(subtoken.text in ["à", "vers"]):
                        destination = token.text
        

    return depart, destination

# Traitement de chaque phrase
for phrase in phrases:
    depart, destination = extraire_localisations(phrase)
    print("Phrase :", phrase)
    print("Départ :", depart)
    print("Destination :", destination)
    print("")
   


Phrase : Je vais de Paris à Londres demain.
Départ : Paris
Destination : Londres

Phrase : Je quitte Bordeaux pour aller à Marseille.
Départ : None
Destination : Marseille

Phrase : Je souhaite me rendre à Lyon depuis Toulouse.
Départ : Toulouse
Destination : Lyon

Phrase : Je mange de la glace à Paris.
Départ : None
Destination : Paris

Phrase : Je marche vers la mer.
Départ : None
Destination : None

Phrase : Je n'ai pas mangé depuis ce matin.
Départ : None
Destination : None



## Conclusion et pistes d'améliorations :

L'algorithme montre une bonne analyse du modèle.
En revanche, comme le montre la phrase "Je quitte Bordeaux pour aller à Marseille." en l'absence de prépositions l'algorithme ne permet pas d'identifier s'il s'agit d'un départ ou d'une destination. De même, pour la phrase "Je mange de la glace à Paris", Paris ne désigne ni un départ ni une destination.

Pour améliorer l'algorithme, nous pouvons étudier les pistes suivantes :
- compléter l'analyse grammaticale en identifiant par exemple une liaison préposition (ou absence de préposition) <--> verbe de déplacement
- entrainer un modèle de machine learning pour reconnaitre des NER customisés de départ et de destination